In [1]:
 # Préparation du fichier preprocessé des indisponibilités des centrales de productionavant fusion avec le fichier des capacités de production

In [2]:
"""
Created on 2024-06-23

@author: Thierry ALLEM
"""

'\nCreated on 2024-06-23\n\n@author: Thierry ALLEM\n'

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import datetime
import matplotlib.pyplot as plt

In [4]:
#==========================  CHARGEMENT des fichiers d'indisponibilités des moyens de production de 2013 à 2022 ===============================================================================
df_result_ind_prod = pd.read_csv('df_result_ind_prod.csv', sep=',',encoding='latin-1')
df_result_ind_prod.info()

# Conversion de la colonne 'date' en datetime
df_result_ind_prod['date_debut'] = pd.to_datetime(df_result_ind_prod['date'])
df_result_ind_prod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67310469 entries, 0 to 67310468
Data columns (total 7 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   region                 object 
 1   filiere_globale        object 
 2   code_eic_centrale      object 
 3   capacite_installee     float64
 4   capacite_disponible    float64
 5   capacite_indisponible  float64
 6   date                   object 
dtypes: float64(3), object(4)
memory usage: 3.5+ GB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67310469 entries, 0 to 67310468
Data columns (total 8 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   region                 object        
 1   filiere_globale        object        
 2   code_eic_centrale      object        
 3   capacite_installee     float64       
 4   capacite_disponible    float64       
 5   capacite_indisponible  float64       
 6   date                   object        
 7   date

In [5]:
df_result_ind_prod.isna().sum()

region                   0
filiere_globale          0
code_eic_centrale        0
capacite_installee       0
capacite_disponible      0
capacite_indisponible    0
date                     0
date_debut               0
dtype: int64

In [6]:
# ===============================RECHERCHE ET TRAITEMENT DES DOUBLONS

In [7]:
doublons = df_result_ind_prod.duplicated(subset=['region', 'filiere_globale', 'code_eic_centrale', 'date']).sum()

print(f"Nombre total de lignes en doublons : {doublons}")

Nombre total de lignes en doublons : 59214038


In [34]:
#    Filtration des doublons et copie dans un nouveau DataFrame
nouveau_dataframe_doublons =df_result_ind_prod[df_result_ind_prod.duplicated(subset=['region', 'filiere_globale', 'code_eic_centrale', 'date'], keep=False)]

In [35]:
#   Export des 1000 premières lignes du DataFrame original
file_export = 'df_result_ind_prod1000.csv'
nouveau_dataframe_doublons.head(10000).to_csv(file_export, index=False)
print("1000 premières lignes du dataframe ind_prod_comp exporté avec succès dans le fichier 'ind_prod_comp1000.csv'.")

1000 premières lignes du dataframe ind_prod_comp exporté avec succès dans le fichier 'ind_prod_comp1000.csv'.


In [36]:
#   Suppression des doublons en ne conservant que la 1ère occurence
df_result_ind_prod_clean = df_result_ind_prod.drop_duplicates(subset=['region', 'filiere_globale', 'code_eic_centrale', 'date'], keep='first')

In [37]:
# Afficher le nombre de lignes après suppression des doublons
print(f"Nombre de lignes après suppression des doublons : {len(df_result_ind_prod_clean)}")

Nombre de lignes après suppression des doublons : 8096431


In [38]:
doublons2 = df_result_ind_prod_clean.duplicated(subset=['region', 'filiere_globale', 'code_eic_centrale', 'date']).sum()

In [39]:
print(f"Nombre total de lignes en doublons après nettoyage: {doublons2}")

Nombre total de lignes en doublons après nettoyage: 0


In [40]:
df_result_ind_prod_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8096431 entries, 0 to 67306212
Data columns (total 8 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   region                 object        
 1   filiere_globale        object        
 2   code_eic_centrale      object        
 3   capacite_installee     float64       
 4   capacite_disponible    float64       
 5   capacite_indisponible  float64       
 6   date                   object        
 7   date_debut             datetime64[ns]
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 555.9+ MB


In [41]:
# Regroupement par 'region', 'filiere_globale' et 'date' en sommant les  valeurs de 'capacite_indisponible' et 'capacite_disponible'

df_result_ind_prod_prefusion = df_result_ind_prod_clean.groupby(['region', 'filiere_globale', 'date'])[['capacite_indisponible']].sum().reset_index()

In [42]:
# Renommage de la colonne des filières
dictionnaire= {'filiere_globale':'filiere'}
df_result_ind_prod_prefusion=df_result_ind_prod_prefusion.rename(dictionnaire, axis = 1)

In [43]:
# Le résultat est un nouveau dataframe 'grouped_df' avec les valeurs sommées
print(df_result_ind_prod_prefusion)

                             region      filiere                 date  \
0              AUVERGNE RHONE ALPES  hydraulique  2013-01-01 17:30:00   
1              AUVERGNE RHONE ALPES  hydraulique  2013-01-01 18:00:00   
2              AUVERGNE RHONE ALPES  hydraulique  2013-01-01 18:30:00   
3              AUVERGNE RHONE ALPES  hydraulique  2013-01-01 19:00:00   
4              AUVERGNE RHONE ALPES  hydraulique  2013-01-04 00:30:00   
...                             ...          ...                  ...   
2424695  PROVENCE ALPES COTE D AZUR    thermique  2022-12-31 21:00:00   
2424696  PROVENCE ALPES COTE D AZUR    thermique  2022-12-31 21:30:00   
2424697  PROVENCE ALPES COTE D AZUR    thermique  2022-12-31 22:00:00   
2424698  PROVENCE ALPES COTE D AZUR    thermique  2022-12-31 22:30:00   
2424699  PROVENCE ALPES COTE D AZUR    thermique  2022-12-31 23:00:00   

         capacite_indisponible  
0                        180.0  
1                        180.0  
2                       

In [44]:
#   Export du dataframe néttoyé et regroupé  qui sera ensuite fusionné avec eco2mix et cap_prod
file_export = 'df_result_ind_prod_prefusion.csv'
df_result_ind_prod_prefusion.to_csv(file_export, index=False, sep = ';')
print("Dataframe 'df_result_ind_prod_fusion' exporté avec succès dans le fichier 'df_result_ind_prod_prefusion.csv'.")

Dataframe 'df_result_ind_prod_fusion' exporté avec succès dans le fichier 'df_result_ind_prod_prefusion.csv'.


In [45]:
# Export des 100 premières lignes dans un nouveau fichier CSV pour aperçu
df_result_ind_prod_prefusion_head = df_result_ind_prod_prefusion.head(100)
df_result_ind_prod_prefusion_head.to_csv('df_result_ind_prod_prefusion_first_100_lines.csv', sep=';', encoding='latin-1', index=False)

In [46]:
print("FIN CODE VENTILATION")

FIN CODE VENTILATION
